____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno sabado mañana__<br/>
__Elaborado por: Santiago Chas__<br/>
__Editado por: Nicolas Aguirre__
____

# Google Colaboratory

In [ ]:
# Verificamos si estamos en Colab
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)
if var_google_colab:
  #Montamos nuestro G.Drive
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  # Direccion root donde está la jupyter-notebook
  root_path = "/content/gdrive/My Drive/Colab Notebooks/ClusterAI/clusterai_2022/clase04/"
  # Direccion donde guardaremos las imagenes
  plot_path = root_path
else:
  # Si, no estamos en google colab, es que estamos corriendo la 
  # en local.
  root_path = ""
  plot_path = root_path  

# Titanic Dataset 

Link: https://www.kaggle.com/c/titanic

## Descripción
|Variable |	Definition |	Key |
| --- | --- | --- |
|survival|	Survival|	0 = No, 1 = Yes|
|pclass|	Ticket class|	1 = 1st, 2 = 2nd, 3 = 3rd|
|sex|	Sex	| |
|Age|	Age in years| |	
|sibsp|	# of siblings / spouses aboard the Titanic| |	
|parch|	# of parents / children aboard the Titanic| |	
|ticket|	Ticket number | |	
|fare|	Passenger fare |	|
|cabin|	Cabin number	| |
|embarked|	Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton

**Objetivo: Clasificar las tres especies de flores segun los datos**

In [ ]:
# Importamos algunas de las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Cargamos el dataset
titanic_df = pd.read_csv(root_path+"train.csv")
# Observamos una parte de los datos
titanic_df.tail(5)

In [ ]:
#Eliminamos columnas que no nos interesan
titanic_df = titanic_df.drop(['PassengerId',"Name", "Ticket","Cabin"],axis=1)
titanic_df.tail(5)

In [ ]:
total = titanic_df.isnull().sum().sort_values(ascending=False)
percent = (titanic_df.isnull().sum()/titanic_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

In [ ]:
list_a = ['S','Q','C']
titanic_df.loc[~titanic_df['Embarked'].isin(list_a),:]

In [ ]:
# Lleno vacíos de age con edad promedio
#titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace = True)
# Lleno Embarked vacíos con "S"
titanic_df['Embarked'].fillna('S', inplace = True)
# chequeamos que todo esta con los NaN
total = titanic_df.isnull().sum().sort_values(ascending=False)
percent = (titanic_df.isnull().sum()/titanic_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

-------------------------------------
# EDA

**Tarea**:

* Imprimir la matriz de correlación

* Histograma y Boxplot por Edad segun "Supervivencia"

* Boxplot para ver Edad en función de la clase (Pclass).


-----------------------------------


# Machine Learning Workflow

In [ ]:
# Prepocessing & Utils.
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
#Metricas
from sklearn.metrics import accuracy_score,roc_curve, auc,confusion_matrix
# Modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Dataset

In [ ]:
# Definimos las variables de entrenamiento y objetivo.
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']
target = ['Survived']
# Generamos X e Y
X = titanic_df.loc[:,features]
Y = titanic_df.loc[:,target]

# Spliteamos Train y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Pipeline y ColumnTransformer


Vamos a continuar con la implementacion del workflow ede trabajo de ML incorporando dos herramientas.

* [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline)


Pipeline aplica secuencialmente una lista de transformaciones y un estimador final. 

Los pasos (*steps*) intermedios de la cadena deben ser "transformadores", es decir, deben implementar los métodos **.fit()** y **transform**. 

* [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

ColumnTransformer aplica transformadores a las columnas de un array o DataFrame de pandas.

Este estimador permite transformar diferentes columnas o subconjuntos de columnas de la entrada por separado y las características generadas por cada transformador se concatenarán para formar un único espacio de características. 

Esto es útil para datos heterogéneos o columnares, para combinar varios mecanismos de extracción de características o transformaciones en un solo transformador.

In [ ]:
# Diferenciemos los tipos de variables (numericas, categoricas)
varaibles_num_1 = ["Age", "Fare"]
varaibles_num_2 = ["SibSp","Parch"]
variables_cat = ["Embarked", "Sex", "Pclass"]

In [ ]:
# Definamos las transformaciones para cada tipo de variable:
transformacion_num_1 = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

transformacion_num_2 = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

transformacion_cat = OneHotEncoder(handle_unknown="error")

In [ ]:
# Juntemos todo:
preprocesamiento = ColumnTransformer(
    # (nombre,transformacion, columnas )
    transformers=[ 
        ("num1", transformacion_num_1, varaibles_num_1),
        ("num2", transformacion_num_2, varaibles_num_2),
        ("cat", transformacion_cat, variables_cat),
    ]
)

 Veamos que nos devuelve hasta aca el preprocesamiento ...

In [ ]:
pd.DataFrame(preprocesamiento.fit_transform(X_train), columns=preprocesamiento.get_feature_names_out())

Bien, ya tenemos el preprocesamiento consolidado ...

Utilicemos nuevamente **Pipeline** para agregarle un estimador ... por ejemplo ... **LogisticRegression**.

In [ ]:
pipeline = Pipeline(
    steps=[("preprocesamiento", preprocesamiento), ("estimador", LogisticRegression())]
)

Workflow hasta el momento:

In [ ]:
from sklearn import set_config
set_config(display="diagram")
pipeline

# PREGUNTAS ?

## Modelos y sus Hyperparametros

Como sabemos podemos querer comparar muchos modelos ...

Por ejemplo, dado un modelo de SVC hariamos ....

```
#Modelo
svc = SVC()
#Pipeline
pipeline = Pipeline(
    steps=[("preprocesamiento", preprocesamiento), ("estimador", svc)]
    )

# Hyperparametros
parameters = {'estimador__kernel':('linear', 'rbf'),
              'estimador__C':[1, 10, 100],
              'estimador__gamma':[0.001, 0.1,1, 10]}
# GridhSearchCV
clf = GridSearchCV(pipeline, # pipeline
                   param_grid = parameters, # Hyperparametros
                   refit = True, # refit nos devuelve el modelo con los mejores parametros encontrados 
                   cv = 5, # cv indica la cantidad de folds
                   verbose=1)
#Fit
clf.fit(X_train, Y_train.values.ravel())

```

Y para un modelo distinto, tendriamso que definir un nuevo modelo, pasarselo a Pipiline ...etc, etc, etc ...

![blablabla](https://i.giphy.com/media/bTvrXKt7qoALMMklz4/giphy.webp)


Pero lo que queremos es poder compactar **todo**:

* Pipeline

* ColumnTransformer

* Models

* GridSearchCV

In [ ]:
# Ya tenemos CASI todo ... solo nos falta definir los modelos y sus hyperparametros

# ATENCION: "__" (son dos "_" seguidos) se usa como indicador de features del estimador. 

parametros = [
    {
        "estimador": (LogisticRegression(),),
          "estimador__C": (0.001,1,10)
    }, 
    {
        "estimador": (RandomForestClassifier(),),
        "estimador__n_estimators": [50, 100], 
        "estimador__max_features": [3, 8], 
        "estimador__max_depth": [5, 20, 50], 
        "estimador__min_samples_leaf":[ 8, 10]
    },
    {
        "estimador": (SVC(),),
          "estimador__kernel":('linear', 'rbf'), 
          "estimador__C":(1, 10, 100), 
          "estimador__gamma":(0.001, 0.01, 0.1,1, 10)          
    }
]

In [ ]:
# Creamos el GSCV: 
grid_search = GridSearchCV(pipeline, parametros,
                  refit = True, # refit nos devuelve el modelo con los mejores parametros encontrados 
                   cv = 5, # cv indica la cantidad de folds
                   verbose=1)

## Fitting !

In [ ]:
grid_search.fit(X_train, Y_train.values.ravel())

In [ ]:
# Toda la informacion del entrenamiento esta en cv_results_
scores = grid_search.cv_results_
#Veamosla ...
scores_df = pd.DataFrame.from_dict(scores)
scores_df.head()

In [ ]:
print("The best parameters are %s with a score of %0.2f" % (grid_search.best_params_, grid_search.best_score_))

## Predict

In [ ]:
pd.DataFrame(preprocesamiento.transform(X_test), columns=preprocesamiento.get_feature_names_out())

In [ ]:
# Prediction sobre las muestras de test
Y_pred = grid_search.predict(X_test)
print(Y_pred)

## Results

In [ ]:
# Computo el accuracy (comparar ytest vs ypred)
test_acc = accuracy_score(Y_test, Y_pred)
print("El accuracy es " + str(test_acc))

In [ ]:
# Compute and plot AUC
Y_proba = grid_search.predict_proba(X_test)
fpr1, tpr1, thresholds = roc_curve(Y_test.astype('int'), Y_proba[:,1], drop_intermediate = False)
auc_value = auc(fpr1, tpr1)
print("El AUC es = " + str(auc_value))

In [ ]:
plt.plot(fpr1, tpr1, lw=2, alpha=0.7 , label = 'ROC curve', color = 'b')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='r', alpha=.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(False)
plt.legend(loc="lower right")
plt.title('ROC Curve')
plt.show()

In [ ]:
# Compute Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
df_cm = pd.DataFrame(cm, index = ['No sobrevivió', 'Sobrevivió'], columns = ['No sobrevivió', "Sobreviviente"])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.title('Confusion matrix')
plt.show()

# PREGUNTAS?

![meme](https://pythonprogramming.net/static/images/svm/machineLearning.png)